# Description
This script filtered from the full wikidata dump politician entitities and dumped useful labels into a compressed json file.
The script works in the pipeline:
1. Load the wikidata dump in stream mode with qwikidata package
2. Go through item's dictionary and filtered data of interest
3. Dump the data into a compressed json file and print debug information ('corrupted fields in wikidata dumps')
The script differs from the provided query_wikidata_dump.py in that 
1. The dumped data only considers alive people.
2. The dumped data only considers politicians.
3. The dumped data provides the start/end data of held positions.
# Required package
[qwikidata](https://qwikidata.readthedocs.io/en/stable/)
[json](https://docs.python.org/3/library/json.html)
[bz2](https://docs.python.org/3/library/bz2.html)
# Required data
[wikidata dump](https://dumps.wikimedia.org/wikidatawiki/entities/)

In [62]:
import time
import json
import bz2
import sys
from qwikidata.entity import WikidataItem
from qwikidata.json_dump import WikidataJsonDump
from qwikidata.utils import dump_entities_to_json

In [148]:
WIKI_DATA_FULL = '../../latest-all.json.bz2' #path to wiki data dump
WIKI_DATA_FILTERED = '../../filtered_politician_v2.json.bz2' #path to filtered wiki data dump
WIKI_DATA_FILTERED_LABELED = '../../filtered_politician_labeled_v2.json.bz2' #path to filtered wikidata with interpretations
WIKI_DATA_FILTERED_MISSINGQ = '../../filtered_politician_missingqids_v2.json.bz2'  #path to missing QIDs in small catalogue gile
#politician ID in wikidata
Q_POLITICIAN = "Q82955" 
#property IDs in wikidata
P_OCCUPATION = "P106"
P_COUNTRY = "P27"
P_POSITION_HELD = "P39"
P_GENDER = "P21"
P_CITIZENSHIP = "P27"
P_RELIGION = "P140"
P_US_CONGRESS_ID = "P1157"
P_CANDIDACY = "P3602"
P_PARTY = "P102"
P_START_TIME = "P580"
P_END_TIME = "P582"
P_DEAD = "P570"

In [3]:
def has_occupation_politician(item: WikidataItem, truthy: bool = True) -> bool:
    """Return True if the Wikidata Item has occupation politician. Adapted from template of qwikidata property query"""
    # Might also be interesting to filter millitary officers (Q189290). (not implemented here)
    if truthy:
        claim_group = item.get_truthy_claim_group(P_OCCUPATION)
    else:
        claim_group = item.get_claim_group(P_OCCUPATION)
    occupation_qids = [
        claim.mainsnak.datavalue.value["id"]
        for claim in claim_group
        if claim.mainsnak.snaktype == "value"
    ]
    return Q_POLITICIAN in occupation_qids #return if Q_POLITICIAN appears in P_OCCUPATION values

In [4]:
# create an instance of WikidataJsonDump
wjd_dump_path = WIKI_DATA_FULL
wjd = WikidataJsonDump(wjd_dump_path)

# create an iterable of WikidataItem representing politicians
politicians = []
t1 = time.time()
with bz2.open(WIKI_DATA_FILTERED, 'wb') as d_file:
    for ii, entity_dict in enumerate(wjd):
        if entity_dict["type"] == "item":
            entity = WikidataItem(entity_dict)
            try:
                if has_occupation_politician(entity):
                    if (P_DEAD not in entity_dict['claims'].keys() 
                        and P_CITIZENSHIP in entity_dict['claims'].keys()):
                        try:
                            temp_position_held = []
                            temp_parties = []
                            temp_usid = ""
                            temp_religion = ""
                            temp_aliases= []
                            temp_candiacy = []
                            #position held and start/end time(optinal field)
                            if (P_POSITION_HELD in entity_dict['claims'].keys()):
                                for v in entity_dict["claims"][P_POSITION_HELD]:
                                    temp_qualifiers = []
                                    if (v["mainsnak"].get("datavalue", {}).get("value", {}).get("id") is not None):
                                        temp_qualifiers.append(v['mainsnak']['datavalue']['value']['id'])#position held
                                        try:
                                            temp_qualifiers.append(v['qualifiers'][P_START_TIME][0]['datavalue']['value']['time'])#start time
                                        except:
                                            temp_qualifiers.append("")
                                        try:
                                            temp_qualifiers.append(v['qualifiers'][P_END_TIME][0]['datavalue']['value']['time'])#start time
                                        except:
                                            temp_qualifiers.append("")
                                    temp_position_held.append(temp_qualifiers)
                            #Party members (optinal field)
                            if (P_PARTY in entity_dict['claims'].keys()):
                                for v in entity_dict["claims"][P_PARTY]:
                                    if (v["mainsnak"].get("datavalue", {}).get("value", {}).get("id") is not None):
                                        temp_parties.append(v['mainsnak']['datavalue']['value']['id']) #parties held **NOTE: there could be people switching parties but wiki doesn't give any temporal information
                            #Candidacy in election (optinal field)
                            if (P_CANDIDACY in entity_dict['claims'].keys()):
                                for v in entity_dict["claims"][P_CANDIDACY]:
                                    if (v["mainsnak"].get("datavalue", {}).get("value", {}).get("id") is not None):
                                        temp_candiacy.append(v['mainsnak']['datavalue']['value']['id']) 
                            # US Congress Bio ID (optinal field)
                            if (P_US_CONGRESS_ID in entity_dict['claims'].keys()):#there are some corrupted data on wikidata with this field but no data
                                if ('datavalue' in entity_dict['claims'][P_US_CONGRESS_ID][0]['mainsnak']):
                                    temp_usid = entity_dict["claims"][P_US_CONGRESS_ID][0]['mainsnak']['datavalue']['value']
                            # Religion (optinal field)
                            if (P_RELIGION in entity_dict['claims'].keys()):
                                temp_religion = entity_dict["claims"][P_RELIGION][0]['mainsnak']['datavalue']['value']['id']
                            #aliases (optional field)
                            
                            if ("aliases" in entity_dict.keys()):
                                if len(entity_dict.get("aliases", {}).get("en", [])) > 0:
                                    for v in entity_dict["aliases"]["en"]:
                                        temp_aliases.append(v["value"])
                            #qid, name, gender, nationality are compulsory fields
                            politicians.append(dict(zip(['qid',
                                                         'name',
                                                         'gender',
                                                         'nationality',
                                                         'aliases',
                                                         'parties',
                                                         'positions held',
                                                         'religion',
                                                         'us_congress_id',
                                                         'candidacy_election'],
                                                        [entity_dict['id'],#Qid
                                                         entity_dict['labels']['en']['value'], #name
                                                         entity_dict['claims'][P_GENDER][-1]['mainsnak']['datavalue']['value']['id'], #latest gender
                                                         entity_dict['claims'][P_CITIZENSHIP][-1]['mainsnak']['datavalue']['value']['id'], #latest country of citizenship
                                                         temp_aliases,
                                                         temp_parties,
                                                         temp_position_held,
                                                         temp_religion,
                                                         temp_usid,
                                                         temp_candiacy
                                                        ])))# dump entity qid, name and nationality only
                        except KeyError as e: #missing important keys (name, qid, country, gender)
                            print(e,entity_dict['id'])
            except Exception:
                print (Exception)
        if ii % 20000 == 0:
            t2 = time.time()
            dt = t2 - t1
            print(
                "Dumped {} politicians among {} entities [entities/s: {:.2f}]".format(
                    len(politicians), ii, ii / dt
                )
            )
            d_file.write(('\n'.join(map(json.dumps, politicians))+'\n').encode('utf-8'))
            politicians = []

Dumped 0 politicians among 0 entities [entities/s: 0.00]
'datavalue' Q76837
Dumped 367 politicians among 20000 entities [entities/s: 1108.58]
'datavalue' Q313915
Dumped 293 politicians among 40000 entities [entities/s: 1206.00]
Dumped 319 politicians among 60000 entities [entities/s: 1346.33]
Dumped 217 politicians among 80000 entities [entities/s: 1457.99]
'en' Q855006
Dumped 193 politicians among 100000 entities [entities/s: 1566.66]
Dumped 176 politicians among 120000 entities [entities/s: 1637.71]
Dumped 179 politicians among 140000 entities [entities/s: 1705.86]
Dumped 270 politicians among 160000 entities [entities/s: 1786.82]
Dumped 255 politicians among 180000 entities [entities/s: 1874.01]
Dumped 346 politicians among 200000 entities [entities/s: 1942.55]
Dumped 179 politicians among 220000 entities [entities/s: 2023.22]
Dumped 168 politicians among 240000 entities [entities/s: 2085.55]
Dumped 188 politicians among 260000 entities [entities/s: 2157.31]
Dumped 117 politicians a

'en' Q10878499
'en' Q10880559
'en' Q10880568
'en' Q10880626
'en' Q10880707
'en' Q10883930
'en' Q10884022
'en' Q10884378
'en' Q10884619
'en' Q10884661
'en' Q10884700
'en' Q10884703
'en' Q10884719
'en' Q10884745
'en' Q10885808
'en' Q10885815
'en' Q10886048
'en' Q10886096
'en' Q10886097
'en' Q10886122
'en' Q10886163
'en' Q10886166
'en' Q10886285
'en' Q10886316
'en' Q10886324
'en' Q10886385
'en' Q10886435
'en' Q10886521
'en' Q10886596
'en' Q10886668
'en' Q10886775
'en' Q10886859
'en' Q10886868
'en' Q10887229
'en' Q10887235
'en' Q10887249
'en' Q10887301
'en' Q10887312
'en' Q10887346
'en' Q10887359
'en' Q10887757
'en' Q10887774
'en' Q10887809
'en' Q10887821
'en' Q10887823
'en' Q10887911
'en' Q10888349
'en' Q10888354
'en' Q10888365
'en' Q10888371
'en' Q10888377
'en' Q10888378
'en' Q10889018
'en' Q10889040
'en' Q10889152
'en' Q10889181
'en' Q10889496
'en' Q10892702
'en' Q10892728
'en' Q10892885
'en' Q10892909
'en' Q10892958
'en' Q10893650
'P21' Q10894541
'en' Q10895260
'en' Q10895262
'en' Q108

'en' Q11146046
'en' Q11146549
'en' Q11146655
'en' Q11147139
'en' Q11147243
'en' Q11149110
'en' Q11151662
'en' Q11172559
'en' Q11172598
'en' Q11172689
'en' Q11173229
'en' Q11174985
'en' Q11175955
'en' Q11175961
'en' Q11176196
'en' Q11176308
'en' Q11176450
'en' Q11176604
'en' Q11176626
'en' Q11176661
'en' Q11176721
'en' Q11176756
'en' Q11177029
'en' Q11177031
'en' Q11177051
'en' Q11177286
'en' Q11177372
'en' Q11177374
'en' Q11177515
'en' Q11177516
'en' Q11177573
'en' Q11178104
'en' Q11178110
'en' Q11178755
'en' Q11178757
'en' Q11178799
'en' Q11178815
'en' Q11178863
'en' Q11180136
'en' Q11180481
'en' Q11180544
'en' Q11180588
'en' Q11180605
'en' Q11180757
'en' Q11180818
'en' Q11180964
'en' Q11181694
'en' Q11271338
'en' Q11274249
Dumped 86 politicians among 1200000 entities [entities/s: 3287.85]
'en' Q11391987
'en' Q11452008
'P21' Q11469353
'P21' Q11471264
'en' Q11477154
'en' Q11477159
'P21' Q11485343
'P21' Q11491129
Dumped 81 politicians among 1220000 entities [entities/s: 3315.94]
'en' Q1

'en' Q15950136
'en' Q15951103
'en' Q15952861
'en' Q15954196
'en' Q15955285
'en' Q15955325
'en' Q15956122
'en' Q15982240
'en' Q16032167
'en' Q16045074
'en' Q16045317
'en' Q16045397
'en' Q16045708
'en' Q16045822
'en' Q16045837
'en' Q16046015
'en' Q16046067
'en' Q16046105
'en' Q16046330
'en' Q16046337
'en' Q16046661
'en' Q16046689
'en' Q16046853
'en' Q16047165
'en' Q16047383
'en' Q16047440
'en' Q16047920
'en' Q16048277
'en' Q16048365
'en' Q16048379
'en' Q16048551
'en' Q16048839
'en' Q16048970
'en' Q16049181
'en' Q16049208
'en' Q16049327
'en' Q16049773
'en' Q16050358
'en' Q16050690
'en' Q16050716
'en' Q16050769
'en' Q16050962
'en' Q16051071
'en' Q16051114
'en' Q16051189
'en' Q16051213
'en' Q16051217
'en' Q16051226
'en' Q16051369
'en' Q16051555
'en' Q16051747
'en' Q16051755
'en' Q16051777
'en' Q16052065
'en' Q16052090
'en' Q16052578
'en' Q16052761
'en' Q16052780
'en' Q16052791
'en' Q16053147
'en' Q16053153
'en' Q16053475
'en' Q16053624
'en' Q16053744
'en' Q16053758
'en' Q16053769
'en' Q1605

'en' Q16949341
'en' Q16949366
'en' Q16949448
'en' Q16949487
'en' Q16949510
'en' Q16949572
'en' Q16949641
'en' Q16949725
'en' Q16949761
'en' Q16949775
'en' Q16949858
'en' Q16951887
'en' Q16978912
'en' Q16978958
Dumped 219 politicians among 1780000 entities [entities/s: 3730.17]
'en' Q17001311
'en' Q17008258
'en' Q17011433
'en' Q17011907
'en' Q17012113
'en' Q17017511
'en' Q17023449
'en' Q17023480
'en' Q17023806
'en' Q17025199
'en' Q17025556
'en' Q17025561
'en' Q17025986
'en' Q17026059
'en' Q17026079
'en' Q17029781
'en' Q17030991
'P21' Q17031934
'en' Q17033085
'en' Q17035927
'en' Q17036168
'en' Q17039879
'en' Q17040326
'en' Q17040379
'en' Q17040781
'en' Q17040853
'P21' Q17060957
'en' Q17157569
'en' Q17163616
'en' Q17164348
'en' Q17164353
Dumped 238 politicians among 1800000 entities [entities/s: 3747.56]
'en' Q17232728
'en' Q17314845
'P21' Q17371016
Dumped 109 politicians among 1820000 entities [entities/s: 3758.17]
'en' Q17401464
'en' Q17499081
'en' Q17500217
'en' Q17500880
Dumped 157 po

...

'en' Q48885674
'en' Q48886531
'en' Q48889564
'en' Q48896407
'en' Q48896930
'P21' Q48911335
'en' Q48919620
'en' Q48926487
'en' Q48926516
'en' Q48926535
'en' Q48927025
'en' Q48927495
'en' Q48927595
'en' Q48927596
'en' Q48927603
'en' Q48928432
'en' Q48928469
'en' Q48928974
'en' Q48929353
'en' Q48929397
'en' Q48929509
'en' Q48929556
'en' Q48929720
'P21' Q48934955
'en' Q48935364
'en' Q48935861
'en' Q48935975
'en' Q48936543
'en' Q48936640
'en' Q48936748
'en' Q48937569
'en' Q48941809
'en' Q48961864
Dumped 19 politicians among 88560000 entities [entities/s: 2810.59]
Dumped 2 politicians among 88580000 entities [entities/s: 2810.73]
'en' Q49249225
'P21' Q49250648
'P21' Q49251727
'P21' Q49252545
'en' Q49253003
'P21' Q49253389
'P21' Q49253488
Dumped 1 politicians among 88600000 entities [entities/s: 2810.97]
Dumped 0 politicians among 88620000 entities [entities/s: 2811.30]
Dumped 1 politicians among 88640000 entities [entities/s: 2811.57]
Dumped 0 politicians among 88660000 entities [entities/s:

Dumped 16 politicians among 89860000 entities [entities/s: 2812.22]
'en' Q61062636
'en' Q61062998
'en' Q61069701
'P21' Q61079477
'P21' Q61079802
'P21' Q61081578
'P21' Q61081646
'en' Q61091238
'en' Q61092025
'en' Q61092990
'en' Q61093292
'en' Q61100871
'en' Q61102745
'en' Q61102788
'en' Q61103100
'en' Q61103114
'en' Q61104683
'en' Q61131819
Dumped 59 politicians among 89880000 entities [entities/s: 2812.56]
Dumped 6 politicians among 89900000 entities [entities/s: 2812.98]
'en' Q61550404
Dumped 34 politicians among 89920000 entities [entities/s: 2813.12]
'en' Q61741206
'en' Q61742528
Dumped 93 politicians among 89940000 entities [entities/s: 2813.38]
'en' Q61863097
Dumped 219 politicians among 89960000 entities [entities/s: 2813.37]
'en' Q61973009
'P21' Q62085383
'en' Q62100346
Dumped 192 politicians among 89980000 entities [entities/s: 2813.59]
Dumped 3 politicians among 90000000 entities [entities/s: 2813.14]
Dumped 27 politicians among 90020000 entities [entities/s: 2813.37]
'en' Q62

Dumped 1 politicians among 90820000 entities [entities/s: 2822.04]
Dumped 4 politicians among 90840000 entities [entities/s: 2822.17]
'en' Q69696502
'en' Q69847378
Dumped 7 politicians among 90860000 entities [entities/s: 2822.32]
'en' Q69856390
'en' Q69856463
'en' Q69856488
'en' Q69992157
Dumped 4 politicians among 90880000 entities [entities/s: 2822.44]
Dumped 0 politicians among 90900000 entities [entities/s: 2822.54]
Dumped 1 politicians among 90920000 entities [entities/s: 2822.67]
Dumped 2 politicians among 90940000 entities [entities/s: 2822.76]
Dumped 6 politicians among 90960000 entities [entities/s: 2822.83]
Dumped 7 politicians among 90980000 entities [entities/s: 2822.94]
'en' Q70869936
'en' Q70876463
Dumped 4 politicians among 91000000 entities [entities/s: 2823.05]
Dumped 2 politicians among 91020000 entities [entities/s: 2823.15]
'en' Q71193850
'en' Q71254729
Dumped 0 politicians among 91040000 entities [entities/s: 2823.07]
Dumped 1 politicians among 91060000 entities [

Dumped 23 politicians among 92800000 entities [entities/s: 2834.70]
Dumped 1 politicians among 92820000 entities [entities/s: 2834.80]
Dumped 1 politicians among 92840000 entities [entities/s: 2834.92]
Dumped 3 politicians among 92860000 entities [entities/s: 2835.04]
Dumped 4 politicians among 92880000 entities [entities/s: 2835.13]
'P21' Q86834248
Dumped 37 politicians among 92900000 entities [entities/s: 2835.22]
Dumped 23 politicians among 92920000 entities [entities/s: 2835.27]
Dumped 6 politicians among 92940000 entities [entities/s: 2835.33]
'en' Q87287761
Dumped 14 politicians among 92960000 entities [entities/s: 2835.39]
Dumped 11 politicians among 92980000 entities [entities/s: 2835.29]
'en' Q87647622
'en' Q87699694
'en' Q87710740
Dumped 1 politicians among 93000000 entities [entities/s: 2835.22]
'en' Q87828276
Dumped 7 politicians among 93020000 entities [entities/s: 2835.33]
'P21' Q87928670
'P21' Q87928761
'P21' Q87928771
'P21' Q87929197
'P21' Q87929295
'P21' Q87959300
'P21

'en' Q100598167
Dumped 9 politicians among 94380000 entities [entities/s: 2839.78]
Dumped 9 politicians among 94400000 entities [entities/s: 2840.01]
'en' Q101095866
'en' Q101096244
'en' Q101096256
'en' Q101096268
'en' Q101096337
'en' Q101096340
'en' Q101096343
Dumped 15 politicians among 94420000 entities [entities/s: 2840.21]
'P21' Q101625971
'P21' Q101626325
Dumped 6 politicians among 94440000 entities [entities/s: 2840.43]
Dumped 6 politicians among 94460000 entities [entities/s: 2840.79]
Dumped 12 politicians among 94480000 entities [entities/s: 2841.16]
Dumped 10 politicians among 94500000 entities [entities/s: 2841.44]
Dumped 3 politicians among 94520000 entities [entities/s: 2841.74]
'en' Q104152235
'en' Q104214567
'en' Q104248340
Dumped 25 politicians among 94540000 entities [entities/s: 2842.05]
'en' Q104383217
Dumped 136 politicians among 94560000 entities [entities/s: 2842.23]
'en' Q104510500
'en' Q104577441
'en' Q104632000
'en' Q104641466
'en' Q104649941
Dumped 48 politici

In [163]:
df = pd.read_json(WIKI_DATA_FILTERED, lines=True, compression='bz2')
print(f'Filtered {len(df)} politician records in total')

Filtered 269337 politician records in total
